# Computation

## General concepts and mechanisms

### Overview

Binary operations between data arrays or datasets behave as follows:

| Property | Action |
| --- | --- |
| coord | compare, abort on mismatch |
| data | apply operation |
| mask | combine with `or` |

### Dimension matching and transposing

Operations "align" variables based on their dimension labels.
That is, an operation between two variables that have a transposed memory layout behave correctly:

In [ ]:
import numpy as np

import scipp as sc

rng = np.random.default_rng(12345)
a = sc.array(
    dims=['x', 'y'], values=rng.random((2, 4)), variances=rng.random((2, 4)), unit='m'
)
b = sc.array(
    dims=['y', 'x'], values=rng.random((4, 2)), variances=rng.random((4, 2)), unit='s'
)
a / b

### Propagation of uncertainties

If variables have variances, operations correctly propagate uncertainties (the variances), in contrast to a naive implementation using NumPy:

In [ ]:
result = a / b
result.values

In [ ]:
a.values / np.transpose(b.values)

In [ ]:
result.variances

In [ ]:
a.variances / np.transpose(b.variances)

The implementation assumes uncorrelated data and is otherwise based on, e.g., [Wikipedia: Propagation of uncertainty](https://en.wikipedia.org/wiki/Propagation_of_uncertainty#Example_formulae).
See also [Propagation of uncertainties](../reference/error-propagation.rst) for the concrete equations used for error propagation.

<div class="alert alert-info">
    <b>Note:</b>

If an operand with variances is also [broadcast](#Broadcasting) in an operation then the resulting values would be correlated.
Scipp has no way of tracking or handling such correlations.
Subsequent operations that combine values of the result, such as computing the mean, would therefore result in *underestimated uncertainties*.

To avoid such silent underestimation of uncertainties, operations raise [VariancesError](https://scipp.github.io/generated/classes/scipp.VariancesError.html) when an operand with variances is implicitly or explicitly broadcast in an operations.
See our publication [Systematic underestimation of uncertainties by widespread neutron-scattering data-reduction software](https://doi.org/10.3233/JNR-220049) for more background.
</div>

### Broadcasting

Missing dimensions in the operands are automatically broadcast.
Consider:

In [ ]:
var_xy = sc.array(dims=['x', 'y'], values=np.arange(6).reshape((2, 3)))
print(var_xy.values)

In [ ]:
var_y = sc.array(dims=['y'], values=np.arange(3))
print(var_y.values)

In [ ]:
var_xy -= var_y
print(var_xy.values)

Since `var_y` did not depend on dimension `'x'` it is considered as "constant" along that dimension.
That is, the *same* `var_y` values are subtracted *from all slices of dimension* `'x'` in `var_xy`.

Given two variables `a` and `b`, we see that broadcasting integrates seamlessly with slicing and transposing:

In [ ]:
a = sc.array(dims=['x', 'y'], values=rng.random((2, 4)), unit='m')
b = sc.array(dims=['y', 'x'], values=rng.random((4, 2)), unit='s')
a.values

In [ ]:
a -= a['x', 1]
a.values

Both operands may be broadcast, creating an output with the combination of input dimensions:

In [ ]:
sc.show(a['x', 1])
sc.show(a['y', 1])
sc.show(a['x', 1] + a['y', 1])

Note that in-place operations such as `+=` will never change the shape of the left-hand-side.
That is only the right-hand-side operation can be broadcast, and the operation fails of a broadcast of the left-hand-side would be required.

### Units

Units are required to be compatible:

In [ ]:
try:
    a + b
except Exception as e:
    print(str(e))

### Coordinate and name matching

In operations with datasets, data items are paired based on their names when applying operations to datasets.
Operations fail if names do not match:

- In-place operations such as `+=` accept a right-hand-side operand that omits items that the left-hand-side has.
  If the right-hand-side contains items that are not in the left-hand-side the operation fails.
- Non-in-place operations such as `+` return a new dataset with items from the intersection of the inputs.

Coordinates are compared in operations with datasets or data arrays (or items of datasets).
Operations fail if there is any mismatch in aligned (see below) coord values.

In [ ]:
d1 = sc.Dataset(
    data={
        'a': sc.array(dims=['x', 'y'], values=rng.random((2, 3))),
        'b': sc.array(dims=['y', 'x'], values=rng.random((3, 2))),
        'c': sc.array(dims=['x', 'y'], values=rng.random((2, 3))),
    },
    coords={'x': sc.arange('x', 2.0, unit='m'), 'y': sc.arange('y', 3.0, unit='m')},
)
d2 = sc.Dataset(
    data={
        'a': sc.array(dims=['x', 'y'], values=rng.random((2, 3))),
        'b': sc.array(dims=['y', 'x'], values=rng.random((3, 2))),
    },
    coords={'x': sc.arange('x', 2.0, unit='m'), 'y': sc.arange('y', 3.0, unit='m')},
)

In [ ]:
d1 += d2

In [ ]:
d2 += d1

In [ ]:
d3 = d1 + d2
for name in d3:
    print(name)

In [ ]:
d3['a'] -= d3['b']  # transposing
d3['a'] -= d3['x', 1]['b']  # broadcasting

In [ ]:
d3['a'] -= d3['x', 1:2]['b']  # fail due to coordinate mismatch

### Alignment

Coordinates are "aligned" be default.
This means that they are required to match between operands and if they don't, an exception is raised.

In [ ]:
da1 = sc.DataArray(
    sc.arange('x', 8).fold('x', sizes={'x': 4, 'y': 2}),
    coords={'x': sc.arange('x', 4), 'y': sc.arange('y', 2)},
)
da2 = sc.DataArray(
    sc.arange('x', 4), coords={'x': 10 * sc.arange('x', 4), 'y': sc.arange('y', 2)}
)

In [ ]:
da1 + da2

Coordinates can become "unaligned" in some operations.
For example, slicing out a single element makes all coordinates in the sliced dimension unaligned:

In [ ]:
da1_sliced = da1['x', 1]
print(da1_sliced.coords['x'].aligned, da1_sliced.coords['y'].aligned)

Range-slices preserve alignment:

In [ ]:
da1['x', 0:1].coords['x'].aligned

Unaligned coordinates are not required to match in operations.
They are instead dropped if there is a mismatch:

In [ ]:
da2_sliced = da2['x', 1]
da1_sliced + da2_sliced

Aligned coordinates take precedence over unaligned if both are present:

In [ ]:
da2.coords.set_aligned('x', False)
da1 + da2

<div class="alert alert-info">
    <b>Note:</b>

`sc.identical` and similar comparison functions ignore alignment when applied to variables directly.
They do, however take it into account when comparing coordinates of data arrays and datasets.
</div>

In [ ]:
da1 = sc.DataArray(sc.arange('x', 4), coords={'x': sc.arange('x', 4)})
da2 = da1.copy()
da2.coords.set_aligned('x', False)

In [ ]:
sc.identical(da1.coords['x'], da2.coords['x'])

In [ ]:
sc.identical(da1, da2)

## Arithmetic

The arithmetic operations `+`, `-`, `*`, and `/` and their in-place variants `+=`, `-=`, `*=`, and `/=` are available for variables, data arrays, and datasets.
They can also be combined with [slicing](slicing.ipynb).

## Trigonometrics

Trigonometric functions like `sin` are supported for variables.
Units for angles provide a safeguard and ensure correct operation when working with either degree or radian:

In [ ]:
rad = 3.141593 * sc.units.rad
deg = 180.0 * sc.units.deg
print(sc.sin(rad))
print(sc.sin(deg))
try:
    rad + deg
except Exception as e:
    print(str(e))

## Other

See the [list of free functions](../reference/free-functions.rst#free-functions) for an overview of other available operations.